In [1]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/lineesha/Documents/GitHub/LLM-Loan-Confidence-Prediction/data/unemployment


In [2]:
import pandas as pd

demographic_data = pd.read_csv('../demographic_data.csv')
difference_data = pd.read_csv('merged_data_unemployment_llama-3.3-70b-versatile.csv.csv')

print('read')

read


In [3]:
print("Demographic columns:\n", demographic_data.columns.tolist())
print("\nDifference columns:\n", difference_data.columns.tolist())

Demographic columns:
 ['County', 'State', 'State FIPS Code', 'County FIPS Code', 'fips', 'Total Population', 'Male Population', 'Female Population', 'Total Race Responses', 'White Alone', 'Black or African American Alone', 'Hispanic or Latino', 'Unnamed: 12', 'White Population', 'Black Population', 'Hispanic/Latino Population', 'Unnamed: 16', 'Unnamed: 17', 'Total']

Difference columns:
 ['fips', 'county', 'unemployment_rate', 'rate_confidence', 'area_unemployment_rate', 'FIPS_Code', 'State', 'Area_Name', 'Attribute', 'Value', 'Unnamed: 10', 'Predicted-Actual']


In [4]:
merged = pd.merge(
    difference_data,
    demographic_data,
    left_on='fips',
    right_on='fips',
    how='inner'
)

In [5]:
#Number of counties with underestimated unemployment rates and higher-than-avg white populations 
merged['difference'] = merged['unemployment_rate'] - merged['Value']

white_avg = 0.598

underestimated = merged[
    (merged['difference'] < 0) &   #predicted < actual
    (merged['White Population'] > white_avg)
]

underestimates = merged[(merged['difference'] < 0)]
num_underestimates = len(underestimates)
print(num_underestimates)

print("Total Counties:", len(merged))
print("Number of underestimated counties with above-average White population:", len(underestimated))
print("Out of total underestimated counties:", num_underestimates)
print("Percentage:", len(underestimated) / num_underestimates * 100)

478
Total Counties: 3130
Number of underestimated counties with above-average White population: 392
Out of total underestimated counties: 478
Percentage: 82.00836820083683


In [6]:
#Number of counties with overestimated unemployment rates and lower-than-avg white populations 
merged['difference'] = merged['unemployment_rate'] - merged['Value']

white_avg = 0.598

overestimated = merged[
    (merged['difference'] > 0) &   # predicted < actual
    (merged['White Population'] < white_avg)
]

overestimates = merged[(merged['difference'] > 0)]
num_overestimates = len(overestimates)

print("Total Counties:", len(merged))
print("Number of overestimated counties with below-average White population:", len(overestimated))
print("Out of total overestimated counties:", num_overestimates)
print("Percentage:", len(overestimated) / num_overestimates * 100)


Total Counties: 3130
Number of overestimated counties with below-average White population: 408
Out of total overestimated counties: 2643
Percentage: 15.43700340522134


In [9]:
#counties with higher than avg white population
higher_white = merged[(merged['White Population']> white_avg)]
print("Percent counties w/ higher than avg white population:", len(higher_white)/len(merged)*100)

lower_white = merged[(merged['White Population']< white_avg)]
print("Percent counties w/ lower than avg white population:", len(lower_white)/len(merged)*100)



Percent counties w/ higher than avg white population: 84.185303514377
Percent counties w/ lower than avg white population: 15.814696485623003
